In [ ]:
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel

import lightgbm as lgbm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
dataset_dir = 'D:/Study/Project/Dataset/songs/'

# 1) load a sample music

In [ ]:
# import IPython.display as ipd
# audiopath = dataset_dir + 'samples/'+'Let It Be (Remastered 2015).mp3'
# test = 'C:/Users/Jason/Downloads/samples/Let It Be (Remastered 2015).mp3'
# ipd.Audio(test)

In [ ]:
from google.colab import drive

# general_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/samples"
general_path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/test200"

def song_names(general_path):
    files = list(os.listdir(f'{general_path}'))
    return files    # filename

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = song_names(general_path)
# filename = filename[0:10]
len(filename)

262

In [ ]:
def load_song(filename):
    song = f'{filename}'
    path = f'{general_path}/{filename}'
    return song, path

In [ ]:
songs = []
paths = []

for file in filename:
    song, path = load_song(file)
    songs.append(f"{song}")
    paths.append(f"{path}")

In [ ]:
import IPython.display as ipd
ipd.Audio(paths[5])
audiopath = paths[5]
ipd.Audio(audiopath)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
! pip install pydub

In [ ]:
from pydub import AudioSegment

# files                                                                       
src = audiopath
dst = filename[5].replace(".mp3","")+".wav"

# convert wav to mp3                                                            
audSeg = AudioSegment.from_file(src, format="mp4")
audSeg.export(dst, format="wav")

<_io.BufferedRandom name='Africa-Toto.wav'>

In [ ]:
song = AudioSegment.from_wav("Africa-Toto.wav")
# Slice audio
# pydub는 milliseconds 단위를 사용한다
thirty_seconds = 30 * 1000
ths = song[:thirty_seconds]
ths.export('result.wav', format='wav', parameters=["-q:a", "10", "-ac", "1"])
smp = "result.wav"

In [ ]:
from pydub import AudioSegment
import os
ths_lists = []
for file in paths:
    dst = file.replace(".mp3","")+".wav"                                                             
    audSeg = AudioSegment.from_file(file, format="mp4")  # convert wav from mp3 
    audSeg.export(dst, format="wav")
    song = AudioSegment.from_wav(dst)
    thirty_seconds = 30 * 1000
    ths = song[:thirty_seconds]
    path = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/test200"
    # dir = "D:/Study/Project/Dataset/songs/test200/_30secs"
    # if not os.path.exists(dir):
    #     os.makedirs(dir)
    name = f'{file.replace(".mp3","").replace(path,"")}_30.wav'
    full_path = path+"/_30secs"+name
    ths.export(full_path, format='wav', parameters=["-q:a", "10", "-ac", "1"])
    ths_lists.append(full_path)

IsADirectoryError: ignored

In [ ]:
len(ths_lists)

In [ ]:
# song = AudioSegment.from_mp3("1.mp3")

# sixty_seconds = 30 * 1000

# for i in range(int(math.floor(len(song)/30000))):
#     slice = song[i*sixty_seconds:sixty_seconds*(i+1)]
#     slice.export('newSong_{}.mp3'.format(i), format="wav")

# Display the waveform of the music:

In [ ]:
sound, sr = librosa.load(smp, duration = 30)
#display waveform
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
plt.figure(figsize=(14, 5))
plt.title('waveform Beatles music')
librosa.display.waveplot(sound, sr=sr)

# Display the Spectrogram:

In [ ]:
X = librosa.stft(sound)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
plt.title('Spectogram Beatles music')
plt.colorbar()

# Features: 

## Zero crossing rate: 
 The number of times the soundwave cross zero

In [ ]:
plt.plot([0,200],[0,0])
plt.plot(sound[9000:9200])

In [ ]:
zero_crossings = librosa.zero_crossings(sound[9000:9200], pad=False)
print(sum(zero_crossings))

## Spectral Centroid: 


It is calculated as the [[weighted mean]] of the frequencies present in the signal, determined using a [[Fourier transform]], with their magnitudes as the weights:

$$
\mathrm{Centroid} = \frac{
  \sum_{n=0}^{N-1}
    f(n)
    x(n)
} {
  \sum_{n=0}^{N-1}
    x(n)
}
$$

where ''x(n)'' represents the weighted frequency value, or magnitude, of [[Histogram|bin]] number ''n'', and ''f(n)'' represents the center frequency of that bin.

<ref>[http://recherche.ircam.fr/equipes/analyse-synthese/peeters/ARTICLES/Peeters_2003_cuidadoaudiofeatures.pdf A Large Set of Audio Features for Sound Description] - technical report published by [[IRCAM]] in 2003. Section 6.1.1 describes the spectral centroid.</ref>

In [ ]:
import sklearn
spectral_centroids = librosa.feature.spectral_centroid(sound, sr=sr)[0]
spectral_centroids.shape
# Computing the time variable for visualization
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)
# Normalising the spectral centroid for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#Plotting the Spectral Centroid along the waveform
librosa.display.waveplot(sound, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='r')

In [ ]:
spectral_centroids

## Rollof frequency: 
The frequency above or below which a filter begins to filter out the harmonics of the waveform.

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(sound, sr=sr)[0]
librosa.display.waveplot(sound, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='r')

## Mel Frequency Cepstral Coefficients (MFCCs):
The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10-20) which concisely describe the overall shape of a spectral envelope. In MIR, it is often used to describe timbre.

In [ ]:
mfccs = librosa.feature.mfcc(sound, sr=sr)
print(mfccs.shape)
librosa.display.specshow(mfccs, sr=sr, x_axis='time')

## spectral contrast:
Spectral contrast considers the spectral peak, the spectral valley, and their difference in each frequency subband.

In [ ]:
spectral_contrast = librosa.feature.spectral_contrast(sound, sr=sr)
spectral_contrast.shape

In [ ]:
plt.imshow(normalize(spectral_contrast, axis=1), aspect='auto', origin='lower', cmap='coolwarm')

## spectral bandwidth:
It computes the order- p  spectral bandwidth:
$$\left( \sum_k S(k) \left(f(k) - f_c \right)^p \right)^{\frac{1}{p}}$$

where  S(k)  is the spectral magnitude at frequency bin  k ,  f(k)  is the frequency at bin  k , and  fc  is the spectral centroid. When  p=2 , this is like a weighted standard deviation.

In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(sound+0.01, sr=sr)[0]
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(sound+0.01, sr=sr, p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(sound+0.01, sr=sr, p=4)[0]
librosa.display.waveplot(sound, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

## Spectral flatness:
Spectral flatness is defined as the ratio of the geometric mean to the arithmetic mean of a power spectrum.

In [ ]:
spectral_flatness = librosa.feature.spectral_flatness(sound)
spectral_flatness.shape

In [ ]:
plt.plot(t, spectral_flatness.reshape(1292,1))
plt.title('Spectral flatness ')
plt.show()

## chroma_stft: 
Compute a chromagram from a waveform or power spectrogram.

# Extract the features:

In [ ]:
def get_features(y, sr):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y)
    features['sample_silence'] = len(y) - len(y_sound)
    
    # Using librosa to calculate the features
    features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
    features['rmse'] = librosa.feature.rms(y).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v) # 첨도(파형의 밀도?)
            result['{}_skew'.format(k)] = skew(v) # 비대칭도
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [ ]:
dir_30sec = "/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/test200/_30secs"

In [ ]:
# def final_extraction(dataset_dir=dir_30sec):
#     genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
#     last_features=[]
#     for genre in genres:
#         path_genre = dataset_dir + '/' + genre 

#         for song in os.listdir(path_genre):        
#             path_song = path_genre + '/' + song
#             y, sr = librosa.load(path_song, duration = 30)
#             features = get_features(y, sr)
#             features['genre'] = genre
#             last_features.append(features)
#     return pd.DataFrame(last_features)

In [ ]:
def final_extraction(dataset_dir=dir_30sec):
    # genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    last_features=[]
    for song in os.listdir(dataset_dir):        
        path_song = dataset_dir + '/' + song
        y, sr = librosa.load(path_song, duration = 30)
        features = get_features(y, sr)
        # features['genre'] = genre
        features['name'] = song.replace(".mp3_30.wav","").replace(".wav","")
        last_features.append(features)
    return pd.DataFrame(last_features)

In [ ]:
%%time

df_features = final_extraction()

In [ ]:
df_features

NameError: ignored

## Export the data to a csv file:

In [ ]:
df_features.to_csv('Data.csv', index=False)

NameError: ignored

## import the data:

In [ ]:
df_feature = pd.read_csv('Data.csv')

In [ ]:
df_names = df_feature['name']
df_names

0                           All By Myself-Eric Carmen_30
1                        Time After Time-Cyndi Lauper_30
2                                     Take On Me-A-ha_30
3                        September-Earth, Wind & Fire_30
4      Piano Man (Album Ver.) (JTBC 드라마 밀회 삽ᄋ...
                             ...                        
126                                   바램-정준일_30
127                                너를 만나-폴킴_30
128    이사랑 (이사랑 버리자)-김영근(지리산소우...
129    서툰 이별을 하려해 (Feat.전상근)-윤토베...
130                         All By Myself-Eric Carmen_30
Name: name, Length: 131, dtype: object

In [ ]:
df_features = df_feature.drop('name', axis = 1)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Columns: 139 entries, centroid_max to tempo
dtypes: float64(137), int64(2)
memory usage: 142.4 KB


In [ ]:
df_features.head()

,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,roloff_std,...,mfcc_11_std,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo
0,4785.856310,0.0,1084.920587,587.201565,13.373900,3.065598,8850.146484,0.0,1982.388013,1172.072252,...,7.153849,0.088902,-0.333404,16.605540,-31.135948,-5.061777,8.033816,0.212790,-0.297490,117.453835
1,3144.461254,0.0,1501.323592,428.617436,2.008232,0.108286,6126.196289,0.0,2912.715721,809.697192,...,6.570369,0.961294,0.158724,6.109903,-32.620705,-11.515361,6.635999,0.047639,-0.337766,117.453835
2,4962.558589,0.0,2398.646708,1120.958269,0.104516,-0.500172,9280.810547,0.0,5134.785617,2174.624365,...,7.956658,1.130822,0.769357,27.810452,-22.702389,-0.014058,7.589836,0.340568,0.399794,172.265625
3,6016.222391,0.0,2332.671608,713.973071,1.626510,0.302899,9194.677734,0.0,5028.219595,1855.441207,...,7.435125,-0.021735,-0.119319,23.976020,-22.039253,-4.289165,6.846467,0.218727,0.376660,123.046875
4,3612.850105,0.0,1753.905891,590.533531,0.239150,0.694835,8225.683594,0.0,3437.037550,1462.874625,...,8.741628,0.597498,0.384927,21.042603,-35.105340,-4.152920,8.383888,-0.110345,-0.189025,172.265625


# Visualization: 

## Dimensionality reduction :

In [ ]:
df_features.corr()

,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,roloff_max,roloff_min,roloff_mean,roloff_std,...,mfcc_11_std,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo
centroid_max,1.000000,-0.035702,0.221557,0.479392,0.166408,0.191657,0.850329,-0.024104,0.225751,0.397495,...,0.102787,-0.062488,-0.005834,0.051266,-0.265523,-0.139830,0.216237,-0.067409,0.010602,-0.051878
centroid_min,-0.035702,1.000000,0.105376,-0.221393,0.039324,0.155039,-0.022251,0.964733,0.068771,-0.234884,...,-0.058304,-0.064306,-0.044113,-0.085169,0.171152,0.057765,-0.136406,-0.115745,0.056904,0.079594
centroid_mean,0.221557,0.105376,1.000000,0.300375,-0.383909,-0.583567,0.224201,0.159006,0.988279,0.165194,...,-0.088381,0.146906,0.118269,-0.041881,0.006839,-0.025539,-0.076108,0.144933,0.115432,0.008622
centroid_std,0.479392,-0.221393,0.300375,1.000000,-0.371392,-0.221474,0.566736,-0.200394,0.336559,0.909672,...,0.102499,-0.012436,0.119528,-0.152012,-0.364902,-0.271673,0.128568,-0.071114,-0.141773,-0.007552
centroid_kurtosis,0.166408,0.039324,-0.383909,-0.371392,1.000000,0.837465,0.101847,0.035124,-0.413294,-0.421645,...,-0.089853,0.010330,-0.011662,0.001941,0.066646,-0.014950,0.031523,-0.053842,0.058253,-0.070416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mfcc_12_mean,-0.139830,0.057765,-0.025539,-0.271673,-0.014950,-0.113582,-0.150155,0.065379,-0.020222,-0.201006,...,0.098256,0.086525,-0.013168,0.576433,0.701560,1.000000,-0.037651,0.121970,0.080464,-0.069287
mfcc_12_std,0.216237,-0.136406,-0.076108,0.128568,0.031523,0.150931,0.244044,-0.168411,-0.110922,0.111340,...,0.650720,0.046213,0.189941,0.524375,-0.485533,-0.037651,1.000000,-0.108964,0.126512,-0.042542
mfcc_12_kurtosis,-0.067409,-0.115745,0.144933,-0.071114,-0.053842,-0.155460,-0.030747,-0.077682,0.142221,-0.061592,...,0.043476,0.670716,0.430660,0.424257,0.056984,0.121970,-0.108964,1.000000,0.414988,0.003082
mfcc_12_skew,0.010602,0.056904,0.115432,-0.141773,0.058253,0.046060,-0.004537,0.048712,0.089507,-0.128984,...,0.199547,0.291823,0.331499,0.527267,0.305074,0.080464,0.126512,0.414988,1.000000,0.088926


In [ ]:
y = df_features['name']
X = df_features.drop(columns=['name'])

In [ ]:
#Scale the data:
scale = StandardScaler()
x_scaled = pd.DataFrame(scale.fit_transform(X), columns = X.columns)

NameError: ignored

In [ ]:
x_scaled.head()

NameError: ignored

In [ ]:
#encode the song_name:
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

NameError: ignored

In [ ]:
# y_enc.reshape(-1, 1)
# y_enc10 = y_enc/10

In [ ]:
pca = PCA(n_components=5, whiten=True)
x_pca = pca.fit_transform(x_scaled)
print("cumulative explained variance ratio = {:.4f}".format(np.sum(pca.explained_variance_ratio_)))

In [ ]:
x_scaled

In [ ]:
# Use LDA only for visualization
lda = LDA()
x_lda = lda.fit_transform(x_scaled, y_enc)

NameError: ignored

In [ ]:
 x_lda.shape

In [ ]:
# Using tsne
tsne = TSNE(n_components=2, verbose=1, learning_rate=250)
x_tsne = tsne.fit_transform(x_scaled)

In [ ]:
plt.figure(figsize=(18, 4))
plt.subplot(131)
plt.scatter(x_pca[:,0], x_pca[:,1], c=y_enc)
plt.colorbar()
plt.title("Embedded space with PCA")

# plt.subplot(132)
# plt.scatter(x_lda[:,0], x_lda[:,1], c=y_enc)
# plt.colorbar()
# plt.title("Embedded space with LDA")

plt.subplot(133)
plt.scatter(x_tsne[:,0], x_tsne[:,1], c=y_enc)
plt.colorbar()
plt.title("Embedded space with TSNE")
plt.show()

In [ ]:
#Prepare the data:
# X_train, X_test, y_train, y_test = train_test_split(x_lda, y_enc, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(x_lda, y_enc, test_size=0.3, random_state=42)

In [ ]:
# First Model: Logistic Regression:
model_ml = 'LogisticRegression'
lg = LogisticRegression(max_iter=2000)
lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)
Accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for this model : '+model_ml+' is :'+str(Accuracy))

In [ ]:
# 2nd model : RandomForestClassifier:
model_ml = 'RandomForest'
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
Accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for this model : '+model_ml+' is :'+str(Accuracy))

In [ ]:
# 3rd model : Kera Neural Network: 
onehot_encoder = OneHotEncoder(sparse=False)
y_ohe_train = y_train.reshape(len(y_train), 1)
y_ohe_train = np.array(onehot_encoder.fit_transform(y_ohe_train))
y_ohe_test = y_test.reshape(len(y_test), 1)
y_ohe_test = np.array(onehot_encoder.fit_transform(y_ohe_test))

#y_ohe_test = np_utils.to_categorical(y_test)
model = tf.keras.Sequential([
layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(9,1)),
layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
layers.Dropout(0.5),
layers.MaxPooling1D(pool_size=3),
layers.Flatten(),
layers.Dense(100, activation='relu'),
layers.Dense(10, activation='softmax')])



model.compile(optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=['AUC','Accuracy','Precision','Recall'])
model.fit(X_train, y_ohe_train,epochs=3)
loss,AUC, Accuracy_test,Precision,Recall = model.evaluate(X_test, y_ohe_test)
F1_score = 2*Precision*Recall/(Precision+Recall)
y_pred = model.predict(X_test)
print('Accuracy is :' + str(Accuracy_test))

In [ ]:
X_train.shape

In [ ]:
Accuracy = accuracy_score(y_test, [np.argmax(y_pred[i]) for i in range(len(y_pred))])

In [ ]:
Accuracy

In [ ]:
nvb = GaussianNB()
nvb.fit(X_train,y_train)
y_pred=nvb.predict(X_test)
#AUC = roc_auc_score(y_test, y_pred)
accuracy_score(y_test, y_pred)